# Turbo Topics Train
## Dependencies

In [1]:
# !git clone https://github.com/blei-lab/turbotopics.git

In [2]:
# !pip install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
import joblib
import os
import re
from tqdm import tqdm

### text preprocessing dependencies
import nltk
from nltk.tokenize.casual import TweetTokenizer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

### sklearn dependencies
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV

### import gensim dependencies
from gensim.models import LdaModel, TfidfModel
from gensim.corpora import Dictionary

[nltk_data] Downloading package wordnet to /home/datallah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
filepath = '/home/datallah/datallah-jaymefis-gibsonce/'
random_state = 42
stop = {'a', 'about', 'above', 'after', 'again', 'against', 'ain',
        'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
        'as', 'at', 'be', 'because', 'been', 'before', 'being',
        'below', 'between', 'both', 'but', 'by', 'can', 'couldn',
        "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does',
        'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during',
        'each', 'few', 'for', 'from', 'further', 'had', 'hadn',
        "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't",
        'having', 'here', 'how', 'i', 'if', 'in', 'into', 'is', 'isn',
        "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm',
        'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn',
        "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor',
        'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or',
        'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
        're', 's', 'same', 'shan', "shan't", 'should', "should've",
        'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than',
        'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves',
        'then', 'there', 'these', 'they', 'this', 'those', 'through',
        'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn',
        "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
        'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won',
        "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll",
        "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}
wnl = WordNetLemmatizer()
lemma_stop_words = [wnl.lemmatize(wrd) for wrd in stop]

## Load & Preprocess

In [5]:
# load df's in dict iterable
def load_df_dict(size = 'one', typ = None):
    """
    Accepts 'one', 'three', or 'five' as input sizes.
    If 'train', 'validate', or 'test' are inserted for type,
    only that kind of sample will be loaded.
    """
    df_dict = {}
    for sample in os.listdir(filepath + 'samples/'):
        if size in sample and (typ is not None or typ in sample):
            name = sample.replace('.csv', '')
            temp_df = pd.read_csv(filepath + 'samples/' + sample)[['source', ' response_text', ' op_gender']]
            temp_df = temp_df.rename(columns = {' response_text': 'response_text', ' op_gender': 'op_gender'}).dropna()
            df_dict[name] = temp_df
    return df_dict

In [6]:
df_dict = load_df_dict(size = 'one', typ = 'train')
print(df_dict.keys())

dict_keys(['validate_one_million', 'test_one_million', 'train_one_million'])


In [7]:
# remove links as these will only apply to specific responses
def rm_link(text):
    return re.sub(r'https?://\S+', '', text)

# remove punctuation
def rm_punct(text):
    return re.sub(r'[^\w\s]', '', text)

# create class that lemmatizes tweet tokens
# this will be used when creating the term matrix
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.tt = TweetTokenizer(preserve_case=False, reduce_len=True,
                                 strip_handles=True, match_phone_numbers=False)
    def __call__(self, docs):
        return [self.wnl.lemmatize(t) for t in self.tt.tokenize(rm_link(rm_punct(docs))) if t not in stop]

# create generic vectorizer fit function
def train_vectorizer(text_data, vectorizer=CountVectorizer, tokenizer=LemmaTokenizer(),
                     ngram_range_lower = 1, ngram_range_upper = 1, min_df = 1):
    """
    Trains a vectorizer on the provided text data and returns the vectorizer instance,
    the document-term matrix, and the feature names.

    Parameters:
    - text_data: List of text documents to be vectorized.
    - vectorizer: Vectorizer class to be used for text vectorization. Defaults to CountVectorizer.
    - tokenizer: Tokenizer class to be used for tokenizing the text documents. Defaults to TweetTokenizer.
    - ngram_range_lower: What's the minimum length of n-grams we want.
    - ngram_range_upper: What's the maximum length of n-grams we want.
    - min_df: Minimum data frequency.

    Returns:
    - instance: The trained vectorizer instance.
    - matrix: The document-term matrix resulting from fitting the vectorizer on `text_data`.
    - features: An array of feature names generated by the vectorizer.
    """
    # Initialize the vectorizer with specified configurations
    instance = vectorizer(
        strip_accents=None,  # Do not strip accents
        lowercase=True,  # Do not convert characters to lowercase
        tokenizer=tokenizer,  # Use the tokenize method of the tokenizer instance
        token_pattern=None,  # Since a tokenizer is provided, token_pattern is not used
        ngram_range=(ngram_range_lower, ngram_range_upper),  # Consider only single words (1-grams)
        min_df=min_df,  # Minimum document frequency for filtering terms
#         max_df=max_df,  # Maximum document frequency for filtering terms
        max_features=None  # No limit on the number of features
    )

    # Fit the vectorizer on the provided text data and transform the data into a matrix
    vector = instance.fit(text_data)
    
    return vector

In [8]:
X_train = df_dict['train_one_million'].response_text
y_train = df_dict['train_one_million'].op_gender
train_tokenized = X_train.apply(LemmaTokenizer())

In [9]:
dictionary = Dictionary(train_tokenized)
train_bow = [dictionary.doc2bow(doc) for doc in train_tokenized]
tfidf_model = TfidfModel(train_bow)
corpus_tfidf = tfidf_model[train_bow]

In [10]:
lda_model = LdaModel(corpus_tfidf,
                     num_topics = 2, 
                     id2word = dictionary,
                     passes = 10,
                     decay = 0.7,
                     offset = 30,
                     random_state = random_state)

In [12]:
top_df = pd.DataFrame(lda_model.get_topics(), columns = lda_model.id2word.values(),
                      index = [f'topic_{i}' for i in range(lda_model.num_topics)]).transpose()
top_df['topic_ind'] = np.where(top_df.topic_0 > top_df.topic_1, 0, 1)
top_df['assign'] = ' ' + top_df.reset_index().index.astype(str) + ':' + top_df.topic_ind.astype(str)
features = top_df.index
assigns = '\n'.join(top_df['assign'])
top_df.sample(10)

,topic_0,topic_1,topic_ind,assign
womankind,1.988433e-06,5.818117e-07,0,75563:0
gained,1.143797e-06,4.695396e-05,1,19616:1
costco,1.307972e-06,8.466465e-06,1,72738:1
possibly,1.082884e-06,1.276737e-04,1,5015:1
transgression,1.057730e-06,1.311828e-06,1,89484:1
edgesocieties,9.974635e-07,5.105277e-07,0,125306:0
definitelyy,1.379649e-06,5.752269e-07,0,145264:0
demure,9.994652e-07,4.302541e-07,0,52336:0
38799,1.847185e-06,1.059828e-06,0,85737:0
artistmusician,1.250159e-06,8.298278e-07,0,164198:0


In [13]:
# create storage path if not exists
tt_path = filepath + 'turbotopics/'
if os.path.exists(tt_path) == False:
    os.mkdir(tt_path)
# save corpus to text file
corpus_path = tt_path + 'corpus.txt'
with open(corpus_path, 'w') as f: f.write('\n'.join(X_train))
# create ngrams file for saved results
out_path = tt_path + "ngrams.txt"
out_file = open(out_path, 'w')
out_file.close()
# save vocab separated by newlines
vocab_path = tt_path + 'vocab.dat'
with open(vocab_path, 'w') as f: f.write('\n'.join(features))
# save index:topic document
assign_path = tt_path + 'assign.dat'
with open(assign_path, 'w') as f: f.write(assigns)
# assign output location
tt_out_path = tt_path + 'tt_result'

In [14]:
%%bash -s "$corpus_path" "$vocab_path" "$assign_path" "$tt_out_path"
cd turbotopics
python2.7 lda_topics.py --corpus="$1" --vocab="$2" --assign="$3" --out="$4" --ntopics=2 --pval=0.001

reading vocabulary from /home/datallah/datallah-jaymefis-gibsonce/turbotopics/vocab.dat
writing topic 0
computing initial counts


Traceback (most recent call last):
  File "lda_topics.py", line 106, in <module>
    pvalue=opt.pvalue)
  File "lda_topics.py", line 75, in turbo_topic
    cnts = tt.nested_sig_bigrams(iter_gen, update_fun, test, min)
  File "/home/datallah/implicit-gender-bias/unsupervised_notebooks/turbotopics/turbotopics.py", line 425, in nested_sig_bigrams
    for doc in iter_generator(): update_fun(counts, doc)
  File "lda_topics.py", line 72, in update_fun
    update_counts_from_topic(doc[0], doc[1], topic, counts)
  File "lda_topics.py", line 65, in update_counts_from_topic
    counts_obj.update_counts(doc, root_filter=root_filter)
  File "/home/datallah/implicit-gender-bias/unsupervised_notebooks/turbotopics/turbotopics.py", line 67, in update_counts
    if (not root_filter(w)): continue
  File "lda_topics.py", line 64, in <lambda>
    root_filter = lambda w: topicmap.get(w.split()[0], -1)==topic
IndexError: list index out of range


CalledProcessError: Command 'b'cd turbotopics\npython2.7 lda_topics.py --corpus="$1" --vocab="$2" --assign="$3" --out="$4" --ntopics=2 --pval=0.001\n'' returned non-zero exit status 1.